In [ ]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # to avoid warning when using a tokenizer to calculate the coherence score during evaluation of a topic model
import sys
import pandas as pd
import pickle
import numpy as np
from sklearn.decomposition import PCA
from bertopic import BERTopic
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from bertopic.representation import KeyBERTInspired
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import Parallel, delayed
from sklearn.metrics import silhouette_score
#from sklearn.cluster import KMeans
from hdbscan import HDBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.model_selection import ParameterGrid
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic
from typing import Optional, Tuple, Dict, Union, List
import datetime
from sklearn.metrics.pairwise import cosine_similarity

# download stopwords and tokenizers
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

# Get the parent directory of the current notebook directory and add it to the python path to import custom modules
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

from util.clustering_utils import *

# Clustering Experiment

## 1. Preparation

#### 1. Load the Dataset

First, we'll load the dataset.

In [ ]:
dataset_path = os.path.join(os.getcwd(), '../data/preprocessed/df_preprocessed.pkl')

with open(dataset_path, 'rb') as f:
    df = pickle.load(f)

df.shape

#### 2. Create Chat-Text-Aggregations for Topic-Interpretability

To ensure the interpretability of the topics generated by BERTopic, we need to provide it with the message texts we used to create features for each chat. 

To make sure that the "chat-text-aggregations" used for this purpose are as meaningful as possible, we will perform the following operations: 

- **Basic preprocessing**, including lowercasing, stop word removal, removal of punctuation and digits, and tokenization.

- **Removing custom stopwords** specific to the Telegram domain, such as:

    - Telegram chat handles, which are frequently used to "sign" each  message in broadcast chats and could distort the analysis of the most common words.

    - Common social media call-to-action phrases, such as "share," "follow," and "comment," which are often repeated irrespective of topic.

- **Multilingual processing:** Since our corpus is multilingual, language-dependent preprocessing will be applied only to messages in the most frequent languages, as the other languages contribute only a marginal number of messages.
- **TF-IDF filtering:** We will filter out words below a certain TF-IDF threshold to ensure that only distinctive terms are included in the aggregation.

Afterwards, we will aggregate the messages and webpage previews for each chat into a single string. This string, along with the chat vector representations created earlier, will be passed to BERTopic as a basis for its topic description.

#### 1. Preprocess Data

**Define the Preprocessing Function**

In [3]:
def preprocess(df: pd.DataFrame, text_column: str) -> pd.DataFrame:
    """
    Preprocess the text data in a specified column of a DataFrame by performing several cleaning operations:
    
    Parameters:
        df (pd.DataFrame): the DataFrame containing the text to preprocess.
        text_column (str): the name of the column that contains the text to clean.

    Returns:
        pd.DataFrame: A DataFrame with additional columns for cleaned (`<text_column>_cleaned`) and tokenized 
        (`<text_column>_preprocessed`) text.
    """

    print("Preprocessing messages...")

    # get stop words 
    stop_words_en = set(stopwords.words('english'))
    stop_words_de = set(stopwords.words('german'))

    cta_stop_words_en = {'click', 'tap', 'press', 'subscribe', 'follow', 'share', 'like', 'comment',
                        'join', 'sign', 'visit', 'download', 'register', 'give', 'message', 'chat', 'group', 'channel', 'bot', 'reply'}
    cta_stop_words_de = {'klicken', 'tippen', 'drücken', 'abonnieren', 'folgen', 'teilen', 'mögen', 'kommentieren',
                        'beitreten', 'anmelden', 'besuchen', 'herunterladen', 'registrieren', 'geben', 'message', 'chat', 'group', 'channel', 'bot', 'reply'}

    stop_words_en = stop_words_en.union(cta_stop_words_en)
    stop_words_de = stop_words_de.union(cta_stop_words_de)
    print("Stop words loaded")

    # get frequent chat handles
    frequent_chat_handles = df["referenced_chat_handles"].explode().value_counts()
    frequent_chat_handles = frequent_chat_handles[frequent_chat_handles > 100].index.tolist()
    print("Frequent chat handles loaded")

    # create regex patterns
    def create_pattern(words):
        return rf'\b(?:{"|".join(map(re.escape, words))})\b'

    frequent_chat_pattern = create_pattern(frequent_chat_handles)
    stop_words_en_pattern = create_pattern(stop_words_en)
    stop_words_de_pattern = create_pattern(stop_words_de)
    print("Regex-Patterns created")
    
    # remove the most frequent chat handles  #TODO: Seems not to work
    df[f"{text_column}_cleaned"] = df[text_column].str.replace(frequent_chat_pattern, '', regex=True, flags=re.IGNORECASE).str.strip()
    print("Handles removed")

    # remove URLs
    pattern = r"(https?:\/\/[^\s/$.?#].[^\s]*[^\s.,?!)](?![\])]))|(www\.[^\s/$.?#].[^\s]*[^\s.,?!)](?![\])]))|(t\.me\/[^\s.,?!)]*)"
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.replace(pattern, '', regex=True).str.strip()
    print("URLs removed")

    # lowercase text
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.lower()
    print("Lowercase")
    
    # remove punctuation
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.replace(f"[{re.escape(string.punctuation)}]", ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    print("Punctuation removed")

    # remove the most frequent chat handles that included an @
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.replace(frequent_chat_pattern, '', regex=True, flags=re.IGNORECASE).str.strip()
    print("Handles with @ removed")

    # remove punctuation again
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.replace(f"[{re.escape(string.punctuation)}]", ' ', regex=True).str.replace(r'\s+', ' ', regex=True).str.strip()
    print("Punctuation removed")

    # remove digits
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].str.replace(r'\d+', '', regex=True).str.strip()
    print("Digits removed")

    # remove english stop words
    df.loc[df["message_text_lang"] == "English", f"{text_column}_cleaned"] = \
        df.loc[df["message_text_lang"] == "English", f"{text_column}_cleaned"].str.replace(stop_words_en_pattern, '', regex=True, flags=re.IGNORECASE).str.strip()
    print("English stop words removed")

    # remove german stop words
    df.loc[df["message_text_lang"] == "German", f"{text_column}_cleaned"] = \
        df.loc[df["message_text_lang"] == "German", f"{text_column}_cleaned"].str.replace(stop_words_de_pattern, '', regex=True, flags=re.IGNORECASE).str.strip()
    print("German stop words removed")

    # fill NaN with empty string
    df[f"{text_column}_cleaned"] = df[f"{text_column}_cleaned"].fillna('')
    
    # tokenize text
    df[f"{text_column}_preprocessed"] = df[f"{text_column}_cleaned"].apply(lambda x: word_tokenize(x) if x else [])

    df[f"{text_column}_preprocessed"] = df[f"{text_column}_preprocessed"].apply(lambda x: ' '.join(x))
    print("Tokenized")

    return df

**Apply Preprocessing to Message Texts**

In [ ]:
# check, if the data was already preprocessed
preprocessen_msg_path = os.path.join(os.getcwd(), '../data/preprocessed/preprocessed_msgs_viz.pkl')
already_preprocessed = os.path.exists(preprocessen_msg_path)
already_preprocessed

In [ ]:
if not already_preprocessed:

    # apply preprocessing
    df = preprocess(df, "message_text")

    # save the preprocessed data
    df["message_text_preprocessed"].to_pickle(preprocessen_msg_path)
    print("Preprocessed messages saved")

else:
    print("Loading preprocessed messages...")
    preprocessed_msg = pd.read_pickle(preprocessen_msg_path)
    df["message_text_preprocessed"] = preprocessed_msg

# display five random samples
with pd.option_context('display.max_colwidth', None):
    display(df[["message_text", "message_text_preprocessed", "referenced_chat_handles"]].sample(5))

**Apply Preprocessing to Webpage Previews**

In [ ]:
# check, if the data was already preprocessed
preprocessen_web_path = os.path.join(os.getcwd(), '../data/preprocessed/preprocessed_web_viz.pkl')
already_preprocessed = os.path.exists(preprocessen_web_path)
already_preprocessed

In [7]:
if not already_preprocessed:

    # apply preprocessing
    df = preprocess(df, "webpage_description")

**Remove Emojis**

In [ ]:
# create a list of emoji-unicodes using data from "https://unicode.org/Public/emoji/15.1/"
if not already_preprocessed:

    # list the paths to the unicode-files
    path_1 = os.path.join(os.getcwd(), "../data/auxiliary/emoji_unicode/emoji-sequences.txt")
    path_2 = os.path.join(os.getcwd(), "../data/auxiliary/emoji_unicode/emoji-test.txt")
    path_3 = os.path.join(os.getcwd(), "../data/auxiliary/emoji_unicode/emoji-zwj-sequences.txt")
    file_paths = [path_1, path_2, path_3]

    # load all emojis from the unicode-files
    emoji_sequences = load_emoji_list(file_paths)

    # create a regex pattern from the emoji sequence
    emoji_pattern = '|'.join(re.escape(emoji) for emoji in emoji_sequences)
    print("Emoji pattern created")

    def demojize_chunk(chunk, emoji_pattern):
        # remove emojis
        chunk["webpage_description_preprocessed"] = chunk["webpage_description_preprocessed"].str.replace(emoji_pattern, " ", regex=True)
        return chunk

    n_jobs = 3  # Use three cores (seems to be fastest?)

    # remove emojis in parallel for each chunk
    chunks = np.array_split(df, n_jobs)
    df_chunks = Parallel(n_jobs=n_jobs)(delayed(demojize_chunk)(chunk, emoji_pattern) for chunk in chunks)
    df = pd.concat(df_chunks, ignore_index=True)

    # save the preprocessed data
    df["webpage_description_preprocessed"].to_pickle(preprocessen_web_path)
    print("Preprocessed messages saved")    

# simply load the preprocessed data, if it was already preprocessed
else:
    print("Loading preprocessed webpage previews...")
    preprocessed_web_previews = pd.read_pickle(preprocessen_web_path)
    df["webpage_description_preprocessed"] = preprocessed_web_previews

#### 2. Apply TF-IDF-Filtering

**Apply TF-IDF-Filtering to Message Texts**

In [ ]:
tfidf_path = os.path.join(os.getcwd(), '../data/preprocessed/tfidf_msgs_viz.pkl')
already_tfidf = os.path.exists(tfidf_path)
already_tfidf

In [ ]:
if not already_tfidf:
    # isolate English and German texts and prepare them for TF-IDF vectorization
    english_texts = df[df["message_text_lang"] == "English"]["message_text_preprocessed"]
    german_texts = df[df["message_text_lang"] == "German"]["message_text_preprocessed"]

    # create and fit TF-IDF vectorizers based on the isolated texts
    tfidf_vectorizer_en = TfidfVectorizer(ngram_range=(1,1))
    tfidf_vectorizer_de = TfidfVectorizer(ngram_range=(1,1)) 
    tfidf_vectorizer_en.fit(english_texts)
    tfidf_vectorizer_de.fit(german_texts)

    feature_names_en = tfidf_vectorizer_en.get_feature_names_out()
    feature_names_de = tfidf_vectorizer_de.get_feature_names_out()

    def apply_tf_idf_threshold(row, tfidf_vectorizer_en, tfidf_vectorizer_de, threshold):

        if row["message_text_lang"] == "English":
            tfidf_vectorizer = tfidf_vectorizer_en
            feature_names = feature_names_en
        elif row["message_text_lang"] == "German":
            tfidf_vectorizer = tfidf_vectorizer_de
            feature_names = feature_names_de
        else:
            return row["message_text_preprocessed"]

        tfidf_matrix = tfidf_vectorizer.transform([row["message_text_preprocessed"]])
        tfidf_values = tfidf_matrix.toarray().flatten()
        
        distinctive_words = [feature_names[i] for i in np.where(tfidf_values > threshold)[0]]

        return ' '.join(distinctive_words)

    # TODO: Change column name
    # apply the threshold to the TF-IDF values
    df["message_text_tfidf"] = df.apply(lambda x: apply_tf_idf_threshold(x, tfidf_vectorizer_en, tfidf_vectorizer_de, 0.15), axis=1)

    # save the preprocessed messages
    df["message_text_tfidf"].to_pickle(tfidf_path)

else:
    print("Loading tf-idf filtered messages...")
    tfidf_filtered_msg = pd.read_pickle(tfidf_path)
    df["message_text_tfidf"] = tfidf_filtered_msg

**Apply TF-IDF-Filtering to Webpage Previews**

In [ ]:
tfidf_path = os.path.join(os.getcwd(), '../data/preprocessed/tfidf_web_viz.pkl')
already_tfidf = os.path.exists(tfidf_path)
already_tfidf

In [ ]:
if not already_tfidf:
    # isolate English and German texts and prepare them for TF-IDF vectorization
    english_texts = df[df["webpage_description_lang"] == "English"]["webpage_description_preprocessed"]
    german_texts = df[df["webpage_description_lang"] == "German"]["webpage_description_preprocessed"]

    # create and fit TF-IDF vectorizers based on the isolated texts
    tfidf_vectorizer_en = TfidfVectorizer(ngram_range=(1,1))
    tfidf_vectorizer_de = TfidfVectorizer(ngram_range=(1,1)) 
    tfidf_vectorizer_en.fit(english_texts)
    tfidf_vectorizer_de.fit(german_texts)

    feature_names_en = tfidf_vectorizer_en.get_feature_names_out()
    feature_names_de = tfidf_vectorizer_de.get_feature_names_out()

    def apply_tf_idf_threshold_web(row, tfidf_vectorizer_en, tfidf_vectorizer_de, threshold):

        if row["webpage_description_lang"] == "English":
            tfidf_vectorizer = tfidf_vectorizer_en
            feature_names = feature_names_en
        elif row["webpage_description_lang"] == "German":
            tfidf_vectorizer = tfidf_vectorizer_de
            feature_names = feature_names_de
        else:
            return row["webpage_description_preprocessed"]

        tfidf_matrix = tfidf_vectorizer.transform([row["webpage_description_preprocessed"]])
        tfidf_values = tfidf_matrix.toarray().flatten()
        
        distinctive_words = [feature_names[i] for i in np.where(tfidf_values > threshold)[0]]

        return ' '.join(distinctive_words)

    # apply the threshold to the TF-IDF values
    df["webpage_description_tfidf"] = df.apply(lambda x: apply_tf_idf_threshold_web(x, tfidf_vectorizer_en, tfidf_vectorizer_de, 0.15), axis=1)

    # save the preprocessed messages
    df["webpage_description_tfidf"].to_pickle(tfidf_path)

else:
    print("Loading tf-idf-filtered messages...")
    tfidf_filtered_msg = pd.read_pickle(tfidf_path)
    df["webpage_description_tfidf"] = tfidf_filtered_msg

#### 3. Create Chat-Text-Aggregations for each Chat

In [ ]:
grouped = df.groupby("telegram_chat_id")
chat_texts = grouped["message_text_tfidf"].agg(lambda x: " ".join(x))
chat_texts = chat_texts.astype(str)
chat_texts

#### 4. Create Webpage-Preview-Aggregations for each Chat

In [ ]:
grouped = df.groupby("telegram_chat_id")
chat_webpage_previews = grouped["webpage_description_tfidf"].agg(lambda x: " ".join(x))
chat_webpage_previews = chat_webpage_previews.astype(str)
chat_webpage_previews

## 2. Basic Chat Embeddings

#### 1. Load Chat Representations

First, we load the chat representations we created in the notebook `02_feature_engineering`

In [ ]:
base_path = os.path.join(os.getcwd(), '../features/0_base_chat_vectors.npy')
base_chat_vectors = np.load(base_path, allow_pickle=True)
print(f"Number of chat vectors: {base_chat_vectors.shape[0]}")
print(f"Vector Dimension: {base_chat_vectors.iloc[0].shape}")
base_chat_vectors

#### 2. Cluster the embeddings

Now we can use BERTopic to cluster the embeddings.

**Define Functions used to run the experiment**

use KeyBERTInspired

Define


In [38]:
# def apply_bertTopic(chat_embeddings: pd.Series, chat_texts: pd.Series, used_embedding_model: SentenceTransformer, hdbscan_model=None, verbose=True): 

#     # prepare the embeddings for dimensionality reduction by stacking them
#     chat_embeddings = np.vstack(chat_embeddings)

#     if verbose:
#         print_log("apply_bertTopic", "Preparation", "Completed ✓")

#     # create your representation model
#     representation_model = KeyBERTInspired() #TODO: Configure?

#     #TODO: Random state?

#     # initiate the BERTopic model
#     docs = chat_texts.tolist()
        
#     #cluster_model = KMeans(n_clusters=14) #9 #15->gut

#     if hdbscan_model is None:
#         topic_model = BERTopic(embedding_model=used_embedding_model, 
#                             verbose=verbose, 
#                             calculate_probabilities=True, 
#                             representation_model=representation_model,
#                             )#hdbscan_model=cluster_model 
#     else:
#         topic_model = BERTopic(embedding_model=used_embedding_model, 
#                             verbose=verbose, 
#                             calculate_probabilities=True, 
#                             representation_model=representation_model,
#                             hdbscan_model=hdbscan_model)

#     if verbose:
#         print_log("apply_bertTopic", "Loading Model", "Completed ✓")
        
#     # fit the model to the reduced embeddings
#     topics, propabilities = topic_model.fit_transform(embeddings = chat_embeddings, documents = docs)
#     if verbose:
#         print_log("apply_bertTopic", "Model Fitting", "Completed ✓")

#     return topics, propabilities, topic_model



# def get_evaluations(chat_embeddings: pd.Series, propabilities: np.ndarray, topic_model: BERTopic, text_aggregations: pd.Series) -> Union[float, float, float, int, int]:
#     """
#     Get several evaluation metrics for a given topic model.
#     Parameters:
#         chat_embeddings (pd.Series): The chat embeddings.
#         propabilities (pd.Series): The propabilities.
#         topic_model (BERTopic): The topic model.
#         text_aggregations (pd.Series): The text aggregations.
#     Returns:
#         tuple: A tuple containing the coherence score, silhouette score, davies bouldin score, topic count and noise count.
#     """
    
#     # get the document info and topics
#     document_info = topic_model.get_document_info(text_aggregations)
#     topics = document_info['Topic']
    
#     # prepare the embeddings by stacking them
#     chat_embeddings = np.vstack(chat_embeddings)    

#     # get the silhouette score while ignoring the "Other" topic
#     valid_indices_filter = topics != -1
#     filtered_embeddings = chat_embeddings[valid_indices_filter]
#     filtered_topics = topics[valid_indices_filter]
#     silhouette_score_result = silhouette_score(X=filtered_embeddings, labels=filtered_topics)
#     #print(f'Silhouette Score: {silhouette_score_result}')
    
#     # get davies bouldin score
#     davies_bouldin_score_result = davies_bouldin_score(X=filtered_embeddings, labels=filtered_topics)
#     #print(f'Davies-Bouldin Score: {davies_bouldin_score_result}')
    
#     # calculate the number of topics found
#     topic_count = len(np.unique(topics))
#     #print(f'Topic Count: {topic_count}')
    
#     # calculate the number of noise points
#     noise_count = len(topics[topics == -1])
#     #print(f'Noise Count: {noise_count}')
    
#     # calculate the coherence score
#     # preprocess documents is not necessary, as we already preprocessed the text aggregations
    
#     # get the vectorizer
#     vectorizer = topic_model.vectorizer_model
#     tokenizer = vectorizer.build_tokenizer()
    
#     # extract features for coherence evaluation
#     #words = vectorizer.get_feature_names()
#     tokens = [tokenizer(doc) for doc in text_aggregations] 
#     dictionary = corpora.Dictionary(tokens)
#     corpus = [dictionary.doc2bow(token) for token in tokens]
#     topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
#                 for topic in range(len(set(topics))-1)]

#     # calculate the coherence score
#     coherence_model = CoherenceModel(topics=topic_words, 
#                                     texts=tokens, 
#                                     corpus=corpus,
#                                     dictionary=dictionary, 
#                                     coherence='c_v') 
#     coherence_score_result = coherence_model.get_coherence()    
    
#     #print(f'Coherence Score: {coherence_score_result}')

#     return coherence_score_result, silhouette_score_result, davies_bouldin_score_result, topic_count, noise_count



# def run_experiment(
#     chat_embeddings: pd.Series, 
#     chat_texts: pd.Series, 
#     n: int, 
#     topic_model_dir_path: str,
#     feature_name: str,
#     used_embedding_model: SentenceTransformer,
#     hdbscan_model: Optional[object] = None
# ) -> Tuple[Dict[str, float], List[int], List[float], BERTopic]:
    
#     """
#     Run the BERTopic model multiple times, calculate average evaluation metrics and return and save the "most average" model for manual inspection.
    
#     Parameters:
#     - chat_embeddings (pd.Series): The chat embeddings.
#     - chat_texts (pd.Series): The aggregated chat texts.
#     - n (int): Number of times to run the model.
#     - topic_model_dir_path (str): Directory path to save topic models.
#     - hdbscan_model (optional): A pre-defined HDBSCAN model to use. If None, a default model will be created.

#     Returns:
#     - avg_evaluation_metrics (dict): A dictionary containing average evaluation metrics.
#     - selected_topics (List[int]): The topics assigned by the most average model.
#     - selected_probabilities (List[float]): The probabilities for each document's topic assignment by the most average model.
#     - avg_model (BERTopic): The BERTopic model that is closest to the average metrics.
#     """
    
#     coherence_scores = []
#     silhouette_scores = []
#     davies_bouldin_scores = []
#     topic_counts = []
#     noise_counts = []
    
#     # save topics and propabilities of each model to return them for the most average model later on
#     topics_list = []
#     propabilities_list = []

#     for i in range(n):
        
#         print(f"\n#### Running Model {i+1}/{n}... ####")
        
#         print_log("run_experiment", "Fitting Model", "Fitting Model")
#         topics, propabilities, topic_model = apply_bertTopic(chat_embeddings, chat_texts, used_embedding_model, hdbscan_model) #Remove hdbscan_model? #TODO: propabilities
#         topics_list.append(topics)
#         propabilities_list.append(propabilities)
#         print_log("run_experiment", "Fitting Model", "Completed ✓")

#         print_log("run_experiment", "Evaluating Model", "Calculating Evaluation Metrics")
#         # calculate evaluation metrics
#         (cs_base_embeddings,
#         ss_base_embeddings,
#         db_base_embeddings,
#         topic_count_base_embeddings, 
#         noise_base_embeddings) = get_evaluations(chat_embeddings, 
#                                                 propabilities, 
#                                                 topic_model, 
#                                                 chat_texts)
        
#         # append the evaluation metrics
#         coherence_scores.append(cs_base_embeddings)
#         silhouette_scores.append(ss_base_embeddings)
#         davies_bouldin_scores.append(db_base_embeddings)
#         topic_counts.append(topic_count_base_embeddings)
#         noise_counts.append(noise_base_embeddings)
#         print_log("run_experiment", "Evaluating Model", "Completed ✓")

#         print_log("run_experiment", "Saving Topic Model", "Saving Model")
#         # save model
#         topic_model_path =  os.path.join(topic_model_dir_path, f"{feature_name}_topic_model_{i}")
#         os.makedirs(topic_model_dir_path, exist_ok=True)
#         topic_model.save(topic_model_path)
#         print_log("run_experiment", "Saving Topic Model", "Completed ✓")

#     print(f"\n#### Calculating Averages ####")
#     print_log("run_experiment", "Calculate Average Evaluation Metrics", "Calculating")    
#     # calculate average evaluation metrics
#     avg_coherence_scores = np.mean(coherence_scores)
#     avg_silhouette_scores = np.mean(silhouette_scores)
#     avg_davies_bouldin_scores = np.mean(davies_bouldin_scores)
#     avg_topic_counts = np.mean(topic_counts)
#     avg_noise_counts = np.mean(noise_counts)
    
#     # create dictionary for the evaluation metrics
#     avg_evaluation_metrics = {
#         "avg_coherence_scores": avg_coherence_scores,
#         "avg_silhouette_scores": avg_silhouette_scores,
#         "avg_davies_bouldin_scores": avg_davies_bouldin_scores,
#         "avg_topic_counts": avg_topic_counts,
#         "avg_noise_counts": avg_noise_counts
#     }
#     print_log("run_experiment", "Calculate Average Evaluation Metrics", "Completed ✓")
    
#     print_log("run_experiment", "Found Most Average Model", "Calculating")
    
#     # find the model with evaluation results closests to the average evaluation metrics
#     # convert evaluation metrics to a matrix for easier comparison with the average value vector
#     evaluation_matrix = np.stack([coherence_scores, silhouette_scores, davies_bouldin_scores, topic_counts, noise_counts])
#     average_value_vector = np.array([avg_coherence_scores, avg_silhouette_scores, avg_davies_bouldin_scores, avg_topic_counts, avg_noise_counts])
#     average_value_vector = average_value_vector[:, np.newaxis]

#     # find the model with the smallest difference to the average evaluation metrics
#     differences = np.abs(evaluation_matrix - average_value_vector)
#     smallest_diff_idx = np.argmin(np.sum(differences, axis=0))
    
#     # load the model with the smallest difference to the average evaluation metrics
#     average_model_path =  os.path.join(topic_model_dir_path, f"{feature_name}_topic_model_{smallest_diff_idx}")
#     avg_model = BERTopic.load(average_model_path)
#     print_log("run_experiment", "Found Most Average Model", "Completed ✓")
    
#     # delete all other models
#     print_log("run_experiment", "Deleted Remaining Models", "Deleting")
#     for i in range(n):
#         if i != smallest_diff_idx:
#             os.remove(os.path.join(topic_model_dir_path, f"{feature_name}_topic_model_{i}"))
            
#     # rename the model 
#     os.rename(average_model_path, os.path.join(topic_model_dir_path, f"avg_{feature_name}_topic_model"))
#     print_log("run_experiment", "Deleted Remaining Models", "Completed ✓")
        
#     return avg_evaluation_metrics, topics[smallest_diff_idx], propabilities[smallest_diff_idx], avg_model

**Run the Experiment**

In [ ]:
# Load the model we used for the embeddings, in order to use it for the representational model
current_path = os.getcwd()
model_dir = os.path.join(current_path, "../data/models/")
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
model_path = os.path.join(model_dir, model_name)

# Load or download the model
if not os.path.isdir(model_path):
    print("Model not found. Downloading...")
    transformer_model = SentenceTransformer(model_name)
    transformer_model.save(model_path)
    print(f"Model saved to {model_path}")
else:
    print(f"Model already downloaded. Loading...")
    transformer_model = SentenceTransformer(model_path)
    
# set the parameters for the HDBSCAN model
alpha = 1.0
min_cluster_size = 7
min_samples = 5
# Running Parameters: {'alpha': 1.0, 'hdbscan_model__min_cluster_size': 7, 'hdbscan_model__min_samples': 5}

hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        alpha=alpha,
                        prediction_data=True)

# run the experiment
base_evaluation_metrics, topics, probabilities, topic_model = run_experiment(
    chat_embeddings=base_chat_vectors, 
    chat_texts=chat_texts, 
    n=50, 
    topic_model_dir_path=os.path.join(os.getcwd(), "../results/base_embeddings/topic_models/"),
    feature_name="base",
    used_embedding_model=transformer_model
)

#### 3. Inspect the average evaluation metrics

In [ ]:
for key, value in base_evaluation_metrics.items():
    print(f"{key.replace("_", " ")[4:].title()}: {value}")

#### 4. Visualise and explore the results of the  most average topic model

In [ ]:
# def create_topic_visualisations(topic_model, embeddings, texts_aggregations):
#     # Visualize topics
#     #topic_model.visualize_topics().show()  

#     print("Topic Map:")
#     # UMAP dimensionality reduction
#     from umap import UMAP
#     import numpy as np
#     docs = texts_aggregations.tolist()
#     embeddings = np.vstack(embeddings)
#     reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
    
#     # Visualize documents using UMAP embeddings
#     topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings).show()

#     print("Bar Chart, displaying the top 13 topics and top 20 words per topic:")
#     # Visualize bar chart for top 13 topics and 20 words per topic
#     topic_model.visualize_barchart(top_n_topics=13, n_words=20).show()

#     print("Hierarchical Topics:")
#     # Visualize hierarchical topics
#     hierarchical_topics = topic_model.hierarchical_topics(texts_aggregations)
#     topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics).show()

create_topic_visualisations(topic_model, base_chat_vectors, chat_texts)

#### 5. Inspect the most representative messages

**1. Create Topic Vectors**

In [ ]:
topics = topic_model.get_topic_info()["Topic"]
topics = topics.values[topics.values != -1]
topics

In [ ]:
#TODO: create!!!
# def create_topic_vectors(topic_model: BERTopic, chat_vectors: pd.Series) -> pd.Series:
#     """ Create a topic vector for each topic by averaging the chat vectors assigned to the topic.

#     Parameters:
#         topic_model (BERTopic): the topic model
#         chat_vectors (pd.Series): the chat vectors used to create the topic vectors

#     Returns:
#         pd.Series: _description_
#     """
    
#     # create a map with chat ids and their corresponding topics
#     topics = topic_model.topics_
#     chat_ids= chat_vectors.index
#     id_topic_map = pd.Series(topics, index=chat_ids, name='Topic')

#     # calculate the mean vector of the chat vectors assigned to each topic
#     topic_vectors = {}
    
#     # iterate over all topics (excluding the "Other" topic)
#     topics = topic_model.get_topic_info()["Topic"]
#     topics = topics.values[topics.values != -1]
#     for topic in topics:
#         topic_chat_vectors = chat_vectors[id_topic_map == topic]
#         topic_vector = np.mean(np.vstack(topic_chat_vectors), axis=0)
#         topic_vectors[topic] = topic_vector
        
#     topic_vectors = pd.Series(topic_vectors, name="Mean_Vector")
    
#     return topic_vectors

topic_vectors = create_topic_vectors(topic_model, base_chat_vectors)
topic_vectors

**2. For each topic, extract the top n messages closest to the topic vector they were assigned to**

First, we add the message-text-embeddings created in `02_feature_engineering` back to the messages in the DataFrame. 
We will then use them to compare the messages to the topic vectors created earlier to find the most representative messages for each topic.

In [ ]:
# load the message embeddings
message_embeddings_path = os.path.join(os.getcwd(), '../features/0_message_embeddings.npy')
message_embeddings = np.load(message_embeddings_path, allow_pickle=True)

# create a series where each element is an message-vector
message_embeddings_series = pd.Series([embedding for embedding in message_embeddings])

# check if the message embeddings have the same shape as the dataframe
assert message_embeddings_series.shape[0] == len(message_embeddings)
message_embeddings_series

# add the message embeddings to the dataframe
df["message_vector"] = message_embeddings_series
df[["message_text", "message_vector"]].head()

In [ ]:
representative_messages = get_representative_texts(df = df,
                                                   topic_model = topic_model,
                                                   topic_vectors = topic_vectors,
                                                   chat_vectors = base_chat_vectors,
                                                   n = 10,
                                                   feature_name = "base",
                                                   text_column = "message_text",
                                                   text_embeddings_column = "message_vector",
                                                   text_preprocessed_column = "message_text_preprocessed")

# save representative messages
import json
representative_messages_path = os.path.join(os.getcwd(), '../results/base_embeddings/representative_messages.json')
representative_messages = {int(topic): messages for topic, messages in representative_messages.items()} # convert keys to int
with open(representative_messages_path, 'w') as jsonfile:
    json.dump(representative_messages, jsonfile, indent=4)

# print representative messages
for topic, messages in representative_messages.items():
    print(f"Topic {topic}:")
    for i, message in enumerate(messages):
        print(f"{i+1}. {message.strip()}")
    print("\n")

## 3. Filtered Chat Embeddings

#### 1. Load the filtered Chat Embeddings

First, we load the chat representations we created in the notebook `02_feature_engineering`

In [ ]:
filtered_path = os.path.join(os.getcwd(), '../features/1_filtered_chat_vectors.npy')
filtered_chat_vectors = np.load(filtered_path, allow_pickle=True)
print(f"Number of chat vectors: {filtered_chat_vectors.shape[0]}")
print(f"Vector Dimension: {filtered_chat_vectors.iloc[0].shape}")
filtered_chat_vectors

#### 2. Create filtered Chat-Message-Text-Aggregations

Now we filter the dataset to remove all Forwarded/Original-Message-Pairs using the indices we saved in  `02_feature_engineering`.

In [ ]:
indices_path = os.path.join(os.getcwd(), "../features/1_implicit_ref_filtered_indices.npy")
filtered_rows_indices = np.load(indices_path)
df_references_filtered = df.loc[filtered_rows_indices]
df_references_filtered.shape

As the dataset already contains the preprocessed Message-Text, we simply need to aggregate the message texts in order to create Chat-Message-Text-Aggregation that exclude Original/Forward-Pairs.

In [ ]:
grouped = df_references_filtered.groupby("telegram_chat_id")
filtered_chat_texts = grouped["message_text_tfidf"].agg(lambda x: " ".join(x))
filtered_chat_texts = filtered_chat_texts.astype(str)
filtered_chat_texts

#### 3. Cluster the embeddings

Now we can use BERTopic to cluster the embeddings. Again, we will run the model multiple times and inspect the average results of the topic models.

In [ ]:
# Load the model we used for the embeddings, in order to reuse it for the representational model
current_path = os.getcwd()
model_dir = os.path.join(current_path, "../data/models/")
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
model_path = os.path.join(model_dir, model_name)

if not os.path.isdir(model_path):
    print("Model not found. Downloading...")
    transformer_model = SentenceTransformer(model_name)
    transformer_model.save(model_path)
    print(f"Model saved to {model_path}")
else:
    print(f"Model already downloaded. Loading...")
    transformer_model = SentenceTransformer(model_path)
    
# set the parameters for the HDBSCAN model
alpha = 1.0
min_cluster_size = 7
min_samples = 5
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        alpha=alpha,
                        prediction_data=True)

# run the experiment
filtered_evaluation_metrics, topics, probabilities, topic_model = run_experiment(
    chat_embeddings=filtered_chat_vectors, 
    chat_texts=filtered_chat_texts, 
    n=50, 
    topic_model_dir_path=os.path.join(os.getcwd(), "../results/filtered_embeddings/topic_models/"),
    feature_name="filtered",
    used_embedding_model=transformer_model
)

#### 3. Inspect the average evaluation metrics

In [ ]:
for key, value in filtered_evaluation_metrics.items():
    print(f"{key.replace("_", " ")[4:].title()}: {value}")

#### 4. Visualise and explore the results

In [ ]:
create_topic_visualisations(topic_model, filtered_chat_vectors, filtered_chat_texts)

#### 5. Inspect the most representative messages

Again, we'll inspect the most representative messages for each topic by comparing them to topic vectors derived from the topic assignments returned by the topic model.

**1. Create Topic Vectors**

In [ ]:
topic_vectors = create_topic_vectors(topic_model, filtered_chat_vectors)
topic_vectors

**2. For each topic, extract the top n messages closest to the topic vector they were assigned to**

Again, we add the embeddings created in `02_feature_engineering` back to the messages in the DataFrame and use them to compare them to the topic vectors created earlier to find the most representative messages for each topic.

In [ ]:
# filter the df to only include the messages that were used in the filtered chat vectors. We can use the indices saved in the feature engineering notebook
filtered_rows_indices = np.load(indices_path)
df_filtered = df.loc[filtered_rows_indices]

# load the message embeddings
message_embeddings_path = os.path.join(os.getcwd(), '../features/0_message_embeddings.npy')
message_embeddings = np.load(message_embeddings_path, allow_pickle=True)

# create a series where each element is a message-vector
message_embeddings_series = pd.Series([embedding for embedding in message_embeddings])

# check if the message embeddings have the same shape as the dataframe
assert message_embeddings_series.shape[0] == len(message_embeddings)
message_embeddings_series

# add the message embeddings to the dataframe
df_filtered["message_vector"] = message_embeddings_series
df_filtered[["message_text", "message_vector"]].head()

In [ ]:
representative_messages = get_representative_texts(df = df_filtered,
                                                   topic_model = topic_model,
                                                   topic_vectors = topic_vectors,
                                                   chat_vectors = filtered_chat_vectors,
                                                   n = 10,
                                                   feature_name = "filtered",
                                                   text_column = "message_text",
                                                   text_embeddings_column = "message_vector",
                                                   text_preprocessed_column = "message_text_preprocessed")

# save representative messages
import json
representative_messages_path = os.path.join(os.getcwd(), '../results/filtered_embeddings/representative_messages.json')
representative_messages = {int(topic): messages for topic, messages in representative_messages.items()} # convert keys to int
with open(representative_messages_path, 'w') as jsonfile:
    json.dump(representative_messages, jsonfile, indent=4)

# print representative messages
for topic, messages in representative_messages.items():
    print(f"Topic {topic}:")
    for i, message in enumerate(messages):
        print(f"{i+1}. {message.strip()}")
    print("\n")

## 4. Webpage Preview Embeddings

#### 1. Load Chat Representations

First, we load the chat representations we created based on webpage previews in the notebook `02_feature_engineering`

In [ ]:
webpreview_path = os.path.join(os.getcwd(), '../features/3_webpreview_chat_vectors.npy')
webpreview_chat_vectors = np.load(webpreview_path, allow_pickle=True)
print(f"Number of chat vectors: {webpreview_chat_vectors.shape[0]}")
print(f"Vector Dimension: {webpreview_chat_vectors.iloc[0].shape}")
webpreview_chat_vectors

#### 2. Cluster the embeddings

Now we can use BERTopic to cluster the embeddings.

In [ ]:
# Load the model we used for the embeddings, in order to use it for the representational model
current_path = os.getcwd()
model_dir = os.path.join(current_path, "../data/models/")
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
model_path = os.path.join(model_dir, model_name)

if not os.path.isdir(model_path):
    print("Model not found. Downloading...")
    transformer_model = SentenceTransformer(model_name)
    transformer_model.save(model_path)
    print(f"Model saved to {model_path}")
else:
    print(f"Model already downloaded. Loading...")
    transformer_model = SentenceTransformer(model_path)
    
# set the parameters for the HDBSCAN model
alpha = 1.0
min_cluster_size = 7
min_samples = 5
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        alpha=alpha,
                        prediction_data=True)

# run the experiment
webpreview_evaluation_metrics, topics, probabilities, topic_model = run_experiment(
    chat_embeddings=webpreview_chat_vectors, 
    chat_texts=chat_webpage_previews, 
    n=50, 
    topic_model_dir_path=os.path.join(os.getcwd(), "../results/webpreview_embeddings/topic_models/"),
    feature_name="webpreview",
    used_embedding_model=transformer_model
)

#### 3. Inspect the average evaluation metrics

In [ ]:
for key, value in webpreview_evaluation_metrics.items():
    print(f"{key.replace("_", " ")[4:].title()}: {value}")

#### 4. Visualize and explore the Results

In [ ]:
create_topic_visualisations(topic_model, webpreview_chat_vectors, chat_webpage_previews)

#### 5. Inspect the most representative messages

**1. Create Topic Vectors**

In [ ]:
topic_vectors = create_topic_vectors(topic_model, webpreview_chat_vectors)
topic_vectors

**2. For each topic, extract the top n messages closest to the topic vector they were assigned to**

Again, we add the embeddings created in `02_feature_engineering` back to the messages in the DataFrame and use them to compare them to the topic vectors created earlier to find the most representative messages for each topic.

In [ ]:
# load the message embeddings
message_embeddings_path = os.path.join(os.getcwd(), '../features/0_message_embeddings.npy')
message_embeddings = np.load(message_embeddings_path, allow_pickle=True)

# create a series where each element is an message-vector
message_embeddings_series = pd.Series([embedding for embedding in message_embeddings])

# check if the message embeddings have the same shape as the dataframe
assert message_embeddings_series.shape[0] == len(message_embeddings)
message_embeddings_series

# add the message embeddings to the dataframe
df["message_vector"] = message_embeddings_series
df[["message_text", "message_vector"]].head()

In [ ]:
representative_messages = get_representative_texts(df,
                                                   topic_model,
                                                   topic_vectors,
                                                   webpreview_chat_vectors,
                                                   10,
                                                   "webpreview",
                                                   "message_text",
                                                   "message_vector",
                                                   "message_text_preprocessed")

# save representative messages
import json
representative_messages_path = os.path.join(os.getcwd(), '../results/webpreview_embeddings/representative_messages.json')
representative_messages = {int(topic): messages for topic, messages in representative_messages.items()} # convert keys to int
with open(representative_messages_path, 'w') as jsonfile:
    json.dump(representative_messages, jsonfile, indent=4)

# print representative messages
for topic, messages in representative_messages.items():
    print(f"Topic {topic}:")
    for i, message in enumerate(messages):
        print(f"{i+1}. {message.strip()}")
    print("\n")

#### Inspect the most representative Webpage-Previews

Now, we will inspect the most representative Webpage-Previews. To do so, we'll again compare the Webpage-Preview-Embeddings to the Topic-Vectors created earlier and inspect the most similar ones.

In [ ]:
# load the webpage-preview embeddings
webpreview_embeddings_path = os.path.join(os.getcwd(), '../features/3_webpage_embeddings.npy')

# create a series where each element is an message-vector
webpreview_embeddings = np.load(webpreview_embeddings_path)

# add the message embeddings to the dataframe.
df["webpreview_vector"] = webpreview_embeddings.tolist() 
df[["webpage_title", "webpage_description", "webpreview_vector"]].head()

In [ ]:
representative_webpreviews = get_representative_texts(df, 
                                                      topic_model, 
                                                      topic_vectors, 
                                                      webpreview_chat_vectors, 
                                                      10, 
                                                      "webpreview", 
                                                      "webpage_description",
                                                      "webpreview_vector",
                                                      "webpage_description_preprocessed")

# save representative messages
import json
representative_webpreview_path = os.path.join(os.getcwd(), '../results/webpreview_embeddings/representative_webpreviews.json')
representative_webpreviews = {int(topic): messages for topic, messages in representative_webpreviews.items()} # convert keys to int
with open(representative_webpreview_path, 'w') as jsonfile:
    json.dump(representative_webpreviews, jsonfile, indent=4)

# print representative messages
for topic, text in representative_webpreviews.items():
    print(f"Topic {topic}:")
    for i, text in enumerate(text):
        print(f"{i+1}. {text.strip()}")
    print("\n")

## 5. Combined Message & Webpage-Preview Embeddings

Next, we combine the two kinds of text embeddings and inspect the changes in clustering results.

#### 1. Combine Message-Text- and Webpage-Preview-Vectors

First, we load the chat-vectors we created by combining the two webpage-preview- and message-vectors by taking their mean.

In [15]:
combine_vectors_path = os.path.join(os.getcwd(), '../features/3_msg_webpreview_chat_vectors.npy')
combined_vectors = np.load(combine_vectors_path, allow_pickle=True)

#### 2. Create combined Chat-Text-Aggregations

Next, we aggregate the Text-Aggregations for Webpage-Previews and Chat-Messages in order to use them  to make the topics interpretable

In [ ]:
# create a DataFrame to combine the texts
combined_text_dataframe = pd.DataFrame({
    "chat_texts": chat_texts,
    "chat_webpage_previews": chat_webpage_previews
})

# combine the texts
combined_text_dataframe["combined_texts"] = combined_text_dataframe["chat_texts"] + " " + combined_text_dataframe["chat_webpage_previews"]

# calculate the length of the texts
combined_text_dataframe["chat_texts_len"] = combined_text_dataframe["chat_texts"].apply(lambda x: len(x.split()))
combined_text_dataframe["chat_webpage_previews_len"] = combined_text_dataframe["chat_webpage_previews"].apply(lambda x: len(x.split()))
combined_text_dataframe["combined_texts_len"] = combined_text_dataframe["combined_texts"].apply(lambda x: len(x.split()))

# check if the combined arrays are the same length as the original arrays combined
assert combined_text_dataframe["combined_texts_len"].equals(combined_text_dataframe["chat_texts_len"] + combined_text_dataframe["chat_webpage_previews_len"])

# get the combined texts
combined_texts = combined_text_dataframe["combined_texts"]
combined_texts

#### 3. Cluster the Combined Embeddings

In [ ]:
# Load the model we used for the embeddings, in order to use it for the representational model
current_path = os.getcwd()
model_dir = os.path.join(current_path, "../data/models/")
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
model_path = os.path.join(model_dir, model_name)

if not os.path.isdir(model_path):
    print("Model not found. Downloading...")
    transformer_model = SentenceTransformer(model_name)
    transformer_model.save(model_path)
    print(f"Model saved to {model_path}")
else:
    print(f"Model already downloaded. Loading...")
    transformer_model = SentenceTransformer(model_path)
    
# set the parameters for the HDBSCAN model
alpha = 1.0
min_cluster_size = 7
min_samples = 5
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size,
                        min_samples=min_samples,
                        alpha=alpha,
                        prediction_data=True)

# run the experiment
combined_webpreview_evaluation_metrics, topics, probabilities, topic_model = run_experiment(
    chat_embeddings=combined_vectors, 
    chat_texts=combined_texts, 
    n=50, 
    topic_model_dir_path=os.path.join(os.getcwd(), "../results/combined_webpreview_embeddings/topic_models/"),
    feature_name="combined_webpreview",
    used_embedding_model=transformer_model
)

#### 4. Inspect the average evaluation metrics

In [ ]:
for key, value in combined_webpreview_evaluation_metrics.items():
    print(f"{key.replace("_", " ")[4:].title()}: {value}")

#### 5. Visualise and explore the Results

In [ ]:
create_topic_visualisations(topic_model, combined_vectors, combined_texts)

#### 6. Inspect the most representative messages

**1. Create Topic Vectors**

In [ ]:
topic_vectors = create_topic_vectors(topic_model, combined_vectors)
topic_vectors

**2. For each topic, extract the top n messages closest to the topic vector they were assigned to**

Like with the features used before, we add the embeddings created in `02_feature_engineering` back to the messages in the DataFrame and use them to compare them to the topic vectors created earlier to find the most representative messages for each topic.

In [ ]:
#TODO: Combine message and webpage embeddings ??? (maybe don't?)

# load the message embeddings
message_embeddings_path = os.path.join(os.getcwd(), '../features/0_message_embeddings.npy')
message_embeddings = np.load(message_embeddings_path, allow_pickle=True)

# create a series where each element is an message-vector
message_embeddings_series = pd.Series([embedding for embedding in message_embeddings])

# check if the message embeddings have the same shape as the dataframe
assert message_embeddings_series.shape[0] == len(message_embeddings)
message_embeddings_series

# add the message embeddings to the dataframe
df["message_vector"] = message_embeddings_series
df[["message_text", "message_vector"]].head()

In [ ]:
representative_messages = get_representative_texts(df = df,
                                                   topic_model = topic_model,
                                                   topic_vectors = topic_vectors,
                                                   chat_vectors = combined_vectors,
                                                   n = 10,
                                                   feature_name = "combined_webpreview",
                                                   text_column = "message_text",
                                                   text_embeddings_column = "message_vector",
                                                   text_preprocessed_column = "message_text_preprocessed")

# save representative messages
import json
representative_messages_path = os.path.join(os.getcwd(), '../results/combined_webpreview_embeddings/representative_messages.json')
representative_messages = {int(topic): messages for topic, messages in representative_messages.items()} # convert keys to int
with open(representative_messages_path, 'w') as jsonfile:
    json.dump(representative_messages, jsonfile, indent=4)

# print representative messages
for topic, messages in representative_messages.items():
    print(f"Topic {topic}:")
    for i, message in enumerate(messages):
        print(f"{i+1}. {message.strip()}")
    print("\n")

#### 7. Inspect the most representative Webpage-Previews

Again, we will inspect the most representative Webpage-Previews using the same approach as before. We will again reuse the topic vectors created while inspecting the most representative messages.

In [ ]:
# load the webpage-preview embeddings
webpreview_embeddings_path = os.path.join(os.getcwd(), '../features/3_webpage_embeddings.npy')

# create a series where each element is an message-vector
webpreview_embeddings = np.load(webpreview_embeddings_path)

# add the message embeddings to the dataframe.
df["webpreview_vector"] = webpreview_embeddings.tolist() 
df[["webpage_title", "webpage_description", "webpreview_vector"]].head()

In [ ]:
representative_webpreviews = get_representative_texts(df, 
                                                      topic_model, 
                                                      topic_vectors, 
                                                      combined_vectors, 
                                                      10, 
                                                      "combined_webpreview", 
                                                      "webpage_description",
                                                      "webpreview_vector",
                                                      "webpage_description_preprocessed")

# save representative webpreviews
import json
representative_webpreview_path = os.path.join(os.getcwd(), '../results/combined_webpreview_embeddings/representative_webpreviews.json')
representative_webpreviews = {int(topic): messages for topic, messages in representative_webpreviews.items()} # convert keys to int
with open(representative_webpreview_path, 'w') as jsonfile:
    json.dump(representative_webpreviews, jsonfile, indent=4)

# print representative webpreviews
for topic, text in representative_webpreviews.items():
    print(f"Topic {topic}:")
    for i, text in enumerate(text):
        print(f"{i+1}. {text.strip()}")
    print("\n")
    
#TODO: What about titles?

## 6. Structural Vectors

Now, we conduct chat-clustering using a chats structural attributes. Structural attributes are a chats connections to other telegram entities. 

For our purposes, we have considered two kinds of connections:

1. Forwarded (fwd) messages between chats.

2. Textual references (ref) to chats or other telegram-entities.

#### 1. Load the Chat-Vectors

To vectorize these connections, we created chat-chat-matrices based on forwards and text based references between chats in `02_feature_engineering`, which we'll load now.

In [41]:
# define paths
fwd_log_path = os.path.join(os.getcwd(), '../features/2_log_fwd_vectors.pkl')
fwd_onehot_path = os.path.join(os.getcwd(), '../features/2_onehot_fwd_vectors.pkl')
ref_log_path = os.path.join(os.getcwd(), '../features/2_log_ref_vectors.pkl')
ref_onehot_path = os.path.join(os.getcwd(), '../features/2_onehot_ref_vectors.pkl')

# load the chat vectors
fwd_log_vectors = pd.read_pickle(fwd_log_path)
fwd_onehot_vectors = pd.read_pickle(fwd_onehot_path)
ref_log_vectors = pd.read_pickle(ref_log_path)
ref_onehot_vectors = pd.read_pickle(ref_onehot_path)



Now we combine the forward-based and the reference-based chat-vectors to create our feature.

In [42]:
# combine the vectors
structure_log_vectors = fwd_log_vectors.combine(ref_log_vectors, lambda x, y: np.concatenate([x,y]))
structure_onehot_vectors = fwd_onehot_vectors.combine(ref_onehot_vectors, lambda x, y: np.concatenate([x,y]))

# check if the combined vectors have the expected length of a sum of the original vectors
dimension_fwd_vectors = len(fwd_log_vectors.iloc[1])
dimension_ref_vectors = len(ref_log_vectors.iloc[1])
assert len(structure_log_vectors.iloc[1]) == dimension_fwd_vectors + dimension_ref_vectors
assert len(structure_onehot_vectors.iloc[1]) == dimension_fwd_vectors + dimension_ref_vectors

#### 2. Cluster the Structural Vectors

Eventhough they are no Text-Embeddings we will pass the structural vectors to BERTopic for clustering. This is possible, as BERTopic can accept any kind of numerical custom embeddings instead of generating them from text.

We will again use the filtered Message-Text-Aggregations to make the topics interpretable. These documents will only be used for topic labeling and interpretation. The clustering itself will be entirely driven by the chat-chat-matrices we pass as embeddings.


In [ ]:
topics, _, topic_model = apply_bertTopic(structure_log_vectors, chat_texts)

#### 3. Visualise and Explore the results

In [ ]:
create_topic_visualisations(topic_model, structure_log_vectors, chat_texts)

#### 4. Evaluate the Results

In [ ]:
(ss_structural_embeddings, 
topic_count_structural_embeddings, 
noise_structural_embeddings) = get_evaluations(structure_log_vectors,
                                               propabilities,
                                               topic_model, 
                                               chat_texts)

## 7. Combined Structural Vectors & Message Embeddings

Now, we conduct chat-clustering using a chats structural attributes. Structural attributes are a chats connections to other telegram entities. 

For our purposes, we have considered two kinds of connections:

1. Forwarded (fwd) messages between chats.

2. Textual references (ref) to chats or other telegram-entities.

#### 1. Load the Chat-Vectors

To vectorize these connections, we created chat-chat-matrices based on forwards and text based references between chats in `02_feature_engineering`, which we'll load now.

In [46]:
# define paths
fwd_log_path = os.path.join(os.getcwd(), '../features/2_log_fwd_vectors.pkl')
fwd_onehot_path = os.path.join(os.getcwd(), '../features/2_onehot_fwd_vectors.pkl')
ref_log_path = os.path.join(os.getcwd(), '../features/2_log_ref_vectors.pkl')
ref_onehot_path = os.path.join(os.getcwd(), '../features/2_onehot_ref_vectors.pkl')

# load the chat vectors
fwd_log_vectors = pd.read_pickle(fwd_log_path)
fwd_onehot_vectors = pd.read_pickle(fwd_onehot_path)
ref_log_vectors = pd.read_pickle(ref_log_path)
ref_onehot_vectors = pd.read_pickle(ref_onehot_path)



Now we combine the forward-based and the reference-based chat-vectors to create our feature.

In [47]:
# combine the vectors
structure_log_vectors = fwd_log_vectors.combine(ref_log_vectors, lambda x, y: np.concatenate([x,y]))
structure_onehot_vectors = fwd_onehot_vectors.combine(ref_onehot_vectors, lambda x, y: np.concatenate([x,y]))

# check if the combined vectors have the expected length of a sum of the original vectors
dimension_fwd_vectors = len(fwd_log_vectors.iloc[1])
dimension_ref_vectors = len(ref_log_vectors.iloc[1])
assert len(structure_log_vectors.iloc[1]) == dimension_fwd_vectors + dimension_ref_vectors
assert len(structure_onehot_vectors.iloc[1]) == dimension_fwd_vectors + dimension_ref_vectors

#### 2. Concenate the Features

To supplement the text embeddings with structural information while still using BERTopic for clustering, we'll concatenate the new feature vectors and the chat embeddings.

In [ ]:
# combine the message embedding based chat vectors with the log-scaled structure-based chat vectors
combined_vectors = base_chat_vectors.combine(structure_log_vectors, lambda x, y: np.concatenate([x, y]))

# check if the combined vectors have the expected length of a sum of the message text embeddings and the structure-based vectors
assert len(combined_vectors.iloc[1]) == len(base_chat_vectors.iloc[1]) + len(structure_log_vectors.iloc[1])

combined_vectors

#### 3. Cluster the Combined Chat Vectors

Now, we can cluster the resulting combined vectors using BERTopic. As we're only use the Messages as textual feature, we will reuse the filtered Chat-Message-Text-Aggregations we created earlier.

In [ ]:
topics, _, topic_model = apply_bertTopic(combined_vectors, chat_texts)

#### 4. Visualise and explore the results

In [ ]:
create_topic_visualisations(topic_model, combined_vectors, chat_texts)

#### 5. Evaluate the Results

In [ ]:
(ss_msg_structural_embeddings, 
 topic_count_msg_structural_embeddings,
 noise_structural_embeddings) = get_evaluations(combined_vectors,
                                                propabilities, 
                                                topic_model, 
                                                chat_texts)

## 8. Evaluation

Finally, we can compare the different approaches based on the evaluation data we collected for each feature and feature combination.

In [ ]:
evaluation_df = pd.DataFrame({
    "Features": ["Msg", "Filtered", "Webpreview", "Msg + Webpreview", "Structural", "Msg + Structural"],
    "Silhouette Score": [ss_base_embeddings, ss_filtered_embeddings, ss_webpreview_embeddings, ss_msg_webpreview_embeddings, ss_structural_embeddings, ss_msg_structural_embeddings],
    "Topic Count": [topic_count_base_embeddings, topic_count_filtered_embeddings, topic_count_webpreview_embeddings, topic_count_msg_webpreview_embeddings, topic_count_structural_embeddings, topic_count_msg_structural_embeddings], 
    "Noise Instances": [noise_base_embeddings, noise_filtered_embeddings, noise_webpreview_embeddings, noise_count_msg_webpreview_embeddings, noise_structural_embeddings, noise_structural_embeddings]
}).set_index("Features")

evaluation_df